# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

600

In [3]:
#debugging only
# cities = cities[0:10]
# cities

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
#create query url to scan API
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"


query_url = f"{base_url}&appid={weather_api_key}&units={units}&q="

In [5]:
#list for analysis parameters
city_name = []
city_lat = []
city_lng = []
city_country =[]
city_date =[]
city_temp = []
city_humidity = []
city_cloud_cover = []
city_wind_speed = []

In [6]:
print("Retrieving Desired City Data")
print(f"----------------------------------")
records = 0
records_set = 1

#for loop to go through each city from the API with try/except block so code doesn't break
for city in cities:
    city_url = f"{query_url}{city}"
    records = records + 1
    try:
        response = requests.get(city_url).json()
        time.sleep(0.5)
        print(f"Processing city number {records}")
        print(response)
        print()
        print()
        city_name.append(city)
        city_lat.append(response["coord"]["lat"])
        city_lng.append(response["coord"]["lon"])
        city_country.append(response["sys"]["country"])
        city_date.append(response["dt"])
        city_temp.append(response["main"]["temp_max"])
        city_humidity.append(response["main"]["humidity"])
        city_cloud_cover.append(response["clouds"]["all"])
        city_wind_speed.append(response["wind"]["speed"])
        
        #Conditional for group city outputs 
        if records > 50:
            records_set += 1
            records = 1
    except:
        print(f"City not found")

print(f"----------------------------------")
print(f"End of Data Retrieval Process")
print(f"----------------------------------")

Retrieving Desired City Data
----------------------------------
Processing city number 1
{'coord': {'lon': 13.5031, 'lat': 68.1214}, 'weather': [{'id': 600, 'main': 'Snow', 'description': 'light snow', 'icon': '13n'}], 'base': 'stations', 'main': {'temp': 32, 'feels_like': 21.22, 'temp_min': 32, 'temp_max': 32, 'pressure': 1026, 'humidity': 80}, 'visibility': 10000, 'wind': {'speed': 11.5, 'deg': 310}, 'snow': {'1h': 0.25}, 'clouds': {'all': 75}, 'dt': 1613167865, 'sys': {'type': 1, 'id': 1642, 'country': 'NO', 'sunrise': 1613115459, 'sunset': 1613142179}, 'timezone': 3600, 'id': 3155152, 'name': 'Gravdal', 'cod': 200}


Processing city number 2
{'coord': {'lon': -41.2439, 'lat': -20.3683}, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 68.58, 'feels_like': 73.11, 'temp_min': 68.58, 'temp_max': 68.58, 'pressure': 1014, 'humidity': 93, 'sea_level': 1014, 'grnd_level': 938}, 'visibility': 10000, 'wind':

Processing city number 17
{'coord': {'lon': 168.3333, 'lat': -46.6}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 62.01, 'feels_like': 61.34, 'temp_min': 62.01, 'temp_max': 62.01, 'pressure': 1015, 'humidity': 68}, 'visibility': 10000, 'wind': {'speed': 1.99, 'deg': 321, 'gust': 8.01}, 'clouds': {'all': 99}, 'dt': 1613167876, 'sys': {'type': 3, 'id': 2008959, 'country': 'NZ', 'sunrise': 1613152636, 'sunset': 1613203482}, 'timezone': 46800, 'id': 2206939, 'name': 'Bluff', 'cod': 200}


Processing city number 18
{'coord': {'lon': -134.9692, 'lat': -23.1203}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 80.31, 'feels_like': 82.56, 'temp_min': 80.31, 'temp_max': 80.31, 'pressure': 1016, 'humidity': 70, 'sea_level': 1016, 'grnd_level': 1013}, 'visibility': 10000, 'wind': {'speed': 9.22, 'deg': 31}, 'clouds': {'all': 30}, '

Processing city number 33
{'coord': {'lon': 102.5, 'lat': 71.9667}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': -27.98, 'feels_like': -39.26, 'temp_min': -27.98, 'temp_max': -27.98, 'pressure': 1028, 'humidity': 85, 'sea_level': 1028, 'grnd_level': 1024}, 'visibility': 5125, 'wind': {'speed': 7.58, 'deg': 199}, 'clouds': {'all': 89}, 'dt': 1613167805, 'sys': {'country': 'RU', 'sunrise': 1613182591, 'sunset': 1613205127}, 'timezone': 25200, 'id': 2022572, 'name': 'Khatanga', 'cod': 200}


Processing city number 34
{'coord': {'lon': -159.319, 'lat': 22.0752}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 77, 'feels_like': 75.2, 'temp_min': 77, 'temp_max': 77, 'pressure': 1018, 'humidity': 53}, 'visibility': 10000, 'wind': {'speed': 8.05, 'deg': 120}, 'clouds': {'all': 1}, 'dt': 1613167888, 'sys': {'type': 1, 'id': 7873, 'countr

Processing city number 49
{'cod': '404', 'message': 'city not found'}


City not found
Processing city number 50
{'coord': {'lon': -73.414, 'lat': 41.3712}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 24.35, 'feels_like': 16, 'temp_min': 23, 'temp_max': 25, 'pressure': 1026, 'humidity': 30}, 'visibility': 10000, 'wind': {'speed': 3.44, 'deg': 70}, 'clouds': {'all': 90}, 'dt': 1613167899, 'sys': {'type': 1, 'id': 3888, 'country': 'US', 'sunrise': 1613130742, 'sunset': 1613168616}, 'timezone': -18000, 'id': 5282297, 'name': 'Bethel', 'cod': 200}


Processing city number 51
{'coord': {'lon': -175.2, 'lat': -21.2}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 84.2, 'feels_like': 89.91, 'temp_min': 84.2, 'temp_max': 84.2, 'pressure': 1014, 'humidity': 79}, 'visibility': 10000, 'wind': {'speed': 10.36, 'deg': 90}, 'clouds': {

Processing city number 17
{'coord': {'lon': -74.8429, 'lat': 1.3349}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 90.63, 'feels_like': 93.69, 'temp_min': 90.63, 'temp_max': 90.63, 'pressure': 1005, 'humidity': 42, 'sea_level': 1005, 'grnd_level': 982}, 'visibility': 10000, 'wind': {'speed': 3.47, 'deg': 73}, 'clouds': {'all': 43}, 'dt': 1613167912, 'sys': {'country': 'CO', 'sunrise': 1613128294, 'sunset': 1613171749}, 'timezone': -18000, 'id': 3792382, 'name': 'Cartagena del Chairá', 'cod': 200}


Processing city number 18
{'coord': {'lon': -173.3, 'lat': 64.3833}, 'weather': [{'id': 600, 'main': 'Snow', 'description': 'light snow', 'icon': '13n'}], 'base': 'stations', 'main': {'temp': -2, 'feels_like': -26.73, 'temp_min': -2, 'temp_max': -2, 'pressure': 1008, 'humidity': 88, 'sea_level': 1008, 'grnd_level': 1007}, 'visibility': 1268, 'wind': {'speed': 32.39, 'deg': 38}, 'snow': {'1h': 0.18}, 'clouds

...

Processing city number 31
{'coord': {'lon': 12.9164, 'lat': 18.6853}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 66.74, 'feels_like': 60.66, 'temp_min': 66.74, 'temp_max': 66.74, 'pressure': 1013, 'humidity': 22, 'sea_level': 1013, 'grnd_level': 972}, 'visibility': 10000, 'wind': {'speed': 3.2, 'deg': 348}, 'clouds': {'all': 16}, 'dt': 1613168243, 'sys': {'country': 'NE', 'sunrise': 1613108272, 'sunset': 1613149647}, 'timezone': 3600, 'id': 2446796, 'name': 'Bilma', 'cod': 200}


Processing city number 32
{'coord': {'lon': 23.6569, 'lat': 35.4939}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 53.19, 'feels_like': 49.64, 'temp_min': 50, 'temp_max': 55.99, 'pressure': 1019, 'humidity': 93}, 'visibility': 10000, 'wind': {'speed': 7.02, 'deg': 253}, 'clouds': {'all': 20}, 'dt': 1613168244, 'sys': {'type': 1, 'id': 6643, 'country':

Processing city number 48
{'coord': {'lon': -89.85, 'lat': 20.55}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 91.18, 'feels_like': 92.8, 'temp_min': 91, 'temp_max': 91.4, 'pressure': 1007, 'humidity': 43}, 'visibility': 10000, 'wind': {'speed': 6.91, 'deg': 170}, 'clouds': {'all': 75}, 'dt': 1613168255, 'sys': {'type': 1, 'id': 7140, 'country': 'MX', 'sunrise': 1613133047, 'sunset': 1613174200}, 'timezone': -21600, 'id': 3522328, 'name': 'Opichen', 'cod': 200}


Processing city number 49
{'coord': {'lon': -53.6735, 'lat': 66.9395}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 13.8, 'feels_like': 3.18, 'temp_min': 13.8, 'temp_max': 13.8, 'pressure': 1005, 'humidity': 92, 'sea_level': 1005, 'grnd_level': 1004}, 'visibility': 2687, 'wind': {'speed': 8.81, 'deg': 136}, 'clouds': {'all': 66}, 'dt': 1613168256, 'sys': {'countr

Processing city number 14
{'coord': {'lon': 110.3333, 'lat': 1.55}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 77, 'feels_like': 85.44, 'temp_min': 77, 'temp_max': 77, 'pressure': 1011, 'humidity': 94}, 'visibility': 10000, 'wind': {'speed': 3.53, 'deg': 355}, 'clouds': {'all': 40}, 'dt': 1613168266, 'sys': {'type': 1, 'id': 9366, 'country': 'MY', 'sunrise': 1613170263, 'sunset': 1613213695}, 'timezone': 28800, 'id': 1735634, 'name': 'Kuching', 'cod': 200}


Processing city number 15
{'cod': '404', 'message': 'city not found'}


City not found
Processing city number 16
{'coord': {'lon': 19.9348, 'lat': 60.0973}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 32, 'feels_like': 21, 'temp_min': 32, 'temp_max': 32, 'pressure': 1032, 'humidity': 74}, 'visibility': 10000, 'wind': {'speed': 11.5, 'deg': 340}, 'clouds': {'all': 0}, 

Processing city number 31
{'cod': '404', 'message': 'city not found'}


City not found
Processing city number 32
{'coord': {'lon': 55.6167, 'lat': -21.3667}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 73.26, 'feels_like': 68.41, 'temp_min': 64.99, 'temp_max': 78.8, 'pressure': 1012, 'humidity': 73}, 'visibility': 10000, 'wind': {'speed': 17.27, 'deg': 110}, 'rain': {'1h': 0.12}, 'clouds': {'all': 0}, 'dt': 1613168279, 'sys': {'type': 1, 'id': 2127, 'country': 'RE', 'sunrise': 1613182013, 'sunset': 1613228209}, 'timezone': 14400, 'id': 6690296, 'name': 'Saint-Joseph', 'cod': 200}


Processing city number 33
{'coord': {'lon': -73.3579, 'lat': 41.1415}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 28.18, 'feels_like': 17.4, 'temp_min': 27, 'temp_max': 30.2, 'pressure': 1026, 'humidity': 31}, 'visibility': 10000, 'wind': {'spee

Processing city number 47
{'coord': {'lon': -8.8033, 'lat': 37.3191}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 52.47, 'feels_like': 49.19, 'temp_min': 52.47, 'temp_max': 52.47, 'pressure': 1027, 'humidity': 83, 'sea_level': 1027, 'grnd_level': 1024}, 'visibility': 10000, 'wind': {'speed': 4.79, 'deg': 18}, 'clouds': {'all': 68}, 'dt': 1613168290, 'sys': {'country': 'PT', 'sunrise': 1613114864, 'sunset': 1613153481}, 'timezone': 0, 'id': 2271968, 'name': 'Aljezur', 'cod': 200}


Processing city number 48
{'coord': {'lon': 76.6333, 'lat': 66.0833}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': -30.21, 'feels_like': -38.83, 'temp_min': -30.21, 'temp_max': -30.21, 'pressure': 1031, 'humidity': 83, 'sea_level': 1031, 'grnd_level': 1023}, 'visibility': 10000, 'wind': {'speed': 2.8, 'deg': 123}, 'clouds': {'all': 100}, 'dt':

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [7]:
#  Get the indices of cities that have humidity over 100%.


In [8]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression